# Making Neural Network Model for Addition


## Data

In [96]:
import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy

In [97]:
def gen_data(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2])        
        y.append(num1+num2)
    return np.array(X), np.array(y)

In [98]:
x_train, y_train = gen_data(10, -50, 50)
x_test, y_test = gen_data(10, 50, 200)

x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,  x_test.shape[1])
[a.shape for a in [x_train, y_train, x_test, y_test]]

[(10, 1, 2), (10,), (10, 1, 2), (10,)]

In [99]:
class RangeAccuracy(Accuracy):
    def __init__(self,arange=(0,1)):
        super(RangeAccuracy,self).__init__(name='RangeAccuracy')
        self.range = arange
    
    def update_state(self, a, b, sample_weight=None):
        valA = super(RangeAccuracy,self).result().numpy()
        valB = super(RangeAccuracy,self).result().numpy()
        if (valA - self.range <= valB and valB <= valA + self.range) or (valB - self.range <= valA and valA <= valB + self.range):
            return super(RangeAccuracy,self).update_state(a,a)
        else:
            return super(RangeAccuracy,self).update_state(a,b)

In [100]:
Acc = RangeAccuracy()

In [101]:
class ModelTester:
    def __init__(self):
        pass
        # test and compare models of different layers... etc...

In [102]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(1,2)),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 1, 128)            384       
_________________________________________________________________
dense_37 (Dense)             (None, 1, 64)             8256      
_________________________________________________________________
dense_38 (Dense)             (None, 1, 1)              65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [103]:
es = EarlyStopping(monitor='val_loss', mode='min')

In [104]:
history = model.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 243ms/step - loss: 4024.3523 - accuracy: 0.0000e+00 - val_loss: 75484.3594 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 56ms/step - loss: 2194.1274 - accuracy: 0.0000e+00 - val_loss: 46240.3203 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 58ms/step - loss: 1343.0696 - accuracy: 0.0000e+00 - val_loss: 28764.8184 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 53ms/step - loss: 834.9658 - accuracy: 0.0000e+00 - val_loss: 17681.0859 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 55ms/step - loss: 512.1620 - accuracy: 0.0000e+00 - val_loss: 10580.2500 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 52ms/step - loss: 305.3519 - accuracy: 0.0000e+00 - val_loss: 6104.3062 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 51ms/step - loss: 1

In [105]:
pred = model.predict(x_test)

In [106]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y}")

input=[193 127] expected_output=[320] prediction=[318.88733] difference=[-1.1126709]
input=[182  50] expected_output=[232] prediction=[230.78809] difference=[-1.2119141]
input=[171  75] expected_output=[246] prediction=[244.90288] difference=[-1.0971222]
input=[ 69 141] expected_output=[210] prediction=[209.61945] difference=[-0.3805542]
input=[153 144] expected_output=[297] prediction=[296.14966] difference=[-0.8503418]
input=[199 107] expected_output=[306] prediction=[304.81134] difference=[-1.1886597]
input=[163 197] expected_output=[360] prediction=[359.2042] difference=[-0.7958069]
input=[ 56 113] expected_output=[169] prediction=[168.63434] difference=[-0.36566162]
input=[ 85 187] expected_output=[272] prediction=[271.6253] difference=[-0.37469482]
input=[156  94] expected_output=[250] prediction=[249.02777] difference=[-0.972229]
